# Churn Prediction

# Imports

In [4]:
import findspark

findspark.init('C:/spark')

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.sql import functions as sqlFunc
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a session

In [6]:
spark = SparkSession.builder.appName('lr_project').getOrCreate()

# Read data

In [7]:
data = spark.read.csv('../../data/customer_churn.csv',
                      inferSchema=True,
                      header=True)

In [8]:
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [9]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



# Drop missing values

In [10]:
data.select([sqlFunc.count(sqlFunc.when(sqlFunc.isnan(c), c)).alias(c) for c in data.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
|    0|  0|             0|              0|    0|        0|           0|       0|      0|    0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+



# Create Pipeline

In [11]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [12]:
feature_cols = ['Age',
                'Total_Purchase',
                'Years',
                'Num_Sites',
                'Company']

In [13]:
feature_data = data.select(feature_cols)

In [14]:
feature_data.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Company: string (nullable = true)



In [15]:
company_indexer = StringIndexer(inputCol='Company',
                                outputCol='CompanyIndexed')
company_encoder = OneHotEncoder(inputCol='CompanyIndexed',
                                outputCol='CompanyVec')

In [16]:
feature_cols.remove('Company')
feature_cols.append('CompanyVec')

In [17]:
assembler = VectorAssembler(inputCols=feature_cols,
                            outputCol='features')

In [18]:
LR_Churn = LogisticRegression(featuresCol='features',
                              labelCol='Churn')

In [19]:
pipeline = Pipeline(stages=[
    company_indexer,
    company_encoder,
    assembler,
    LR_Churn
])

# Train test split

In [20]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [21]:
fit_model = pipeline.fit(train_data)

In [22]:
predictions = fit_model.transform(test_data)

In [23]:
predictions.select(['Churn']).show()

+-----+
|Churn|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    1|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    1|
|    0|
|    0|
+-----+
only showing top 20 rows



# Model evaluation

In [24]:
evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [27]:
AUC = evaluation.evaluate(predictions)

Py4JJavaError: An error occurred while calling o351.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 53.0 failed 1 times, most recent failure: Lost task 0.0 in stage 53.0 (TID 47) (DESKTOP-VGJQVC9 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3326/319358669: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at scala.collection.Iterator$$a